# Table of Contents
 <p><div class="lev1"><a href="#Preprocessing"><span class="toc-item-num">1&nbsp;&nbsp;</span>Preprocessing</a></div><div class="lev2"><a href="#Imports-and-loading-the-data"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Imports and loading the data</a></div><div class="lev2"><a href="#Cleaning-the-data"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Cleaning the data</a></div><div class="lev3"><a href="#Remove-constant-a-duplicate-columns"><span class="toc-item-num">1.2.1&nbsp;&nbsp;</span>Remove constant a duplicate columns</a></div><div class="lev3"><a href="#Save-the-IDs-and-TARGETs-and-drop-them-from-the-dataframe"><span class="toc-item-num">1.2.2&nbsp;&nbsp;</span>Save the IDs and TARGETs and drop them from the dataframe</a></div><div class="lev3"><a href="#Pay-close-attention-to-the-important-features"><span class="toc-item-num">1.2.3&nbsp;&nbsp;</span>Pay close attention to the important features</a></div><div class="lev1"><a href="#Feature-grooming"><span class="toc-item-num">2&nbsp;&nbsp;</span>Feature grooming</a></div><div class="lev2"><a href="#var38"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>var38</a></div>

# Preprocessing
## Imports and loading the data

In [ ]:
%matplotlib inline
from __future__ import division

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# Input data files are available in the "./input/" directory.
# load data
df_train = pd.read_csv('../input/train.csv')
df_test = pd.read_csv('../input/test.csv')

# load feature importances generated form "random-forest.ipynb"
df_features = pd.read_csv('../misc/features_random_forest_20160408_062851.csv')

## Cleaning the data
### Remove constant a duplicate columns
We remove any constant columns and any duplicated columns (identical values) as these can have no signature in the dependent variable. Note that we remove the constant and duplicate columns in the training set **and the test set**.

In [ ]:
# remove constant columns
remove = []
for col in df_train.columns:
    if df_train[col].std() == 0:
        remove.append(col)

df_train.drop(remove, axis=1, inplace=True)
df_test.drop(remove, axis=1, inplace=True)

# remove duplicated columns
remove = []
c = df_train.columns
for i in range(len(c)-1):
    v = df_train[c[i]].values
    for j in range(i+1,len(c)):
        if np.array_equal(v,df_train[c[j]].values):
            remove.append(c[j])

df_train.drop(remove, axis=1, inplace=True)
df_test.drop(remove, axis=1, inplace=True)

### Save the IDs and TARGETs and drop them from the dataframe

In [ ]:
IDs = df_train["ID"]
IDs_test = df_test["ID"]
TARGETs = df_train["TARGET"]

df_train.drop(["ID", "TARGET"], axis=1, inplace=True)
df_test.drop(["ID"], axis=1, inplace=True)

### Pay close attention to the important features

We now examine the questions:
1. Do the important features have missing values?
2. Can any of the important features be split?

As an example of the second question, if a feature takes a "special" value many more times than any other value, that feature can be split into two features:
1. A binary feature (0/1), representing whether or not a record takes the "special"
2. A feature that equals the original feature unless the special value obtains, in which case it is zero. 
Below, we show that "var38", which ranked top in our random forest classification can be split into two features. Since it ranked so highly, this may be worth our time.

To try to answer questions 1. and 2., we examine the value counts for the top 'n_features' features. This can give us hints about missing values (extreme outliers) as well as suggestions about splitting variables.

In [ ]:
n_features = 40
df_features_keep = df_features.iloc[0:n_features,:]

df_train_keep = df_train[df_features_keep["Feature"]]


for i in range(0,n_features):
    print(df_train_keep[df_features_keep["Feature"][i]].value_counts().iloc[0:5])

For the following discussion, we go up to "n_features" = 40. Carefully looking through the value counts we find:
1. var38: should be split in the manner described above
2. num_var22_hace2: this feature obtains values 0,1,2,3,99. With many entries equal to 99. I expect that 99 is a missing value. We can deal with this by splitting in the manner described above.
3. imp_op_var_39_efect_ult3: This feature has
    * a clear outlier, -999999, almost certainly representing missing value. Since not so many are missing, we can probably safely replace -999999 --> 2 (the most common value)
    * a dominant mode (2). This variable can be split in the manner described above.

We also note that lots of variables have zero as the dominant mode. Some of these should perhaps be split, but without knowing the meaning of the variables, it's hard to know.

# Feature grooming
We will now implement the changes suggested in points 1 through 3 above.

## var38

In [ ]:
mode = df_train_keep.var38.mode()
df_train_keep['var38mode']=np.isclose(df_train.var38, mode)
print(df_train_keep['var38mode'].value_counts())

df_train_keep.loc[df_train_keep["var38mode"], "var38"]=0
print(df_train_keep['var38'].value_counts().iloc[0:10])